# 【新手入门】PaddleX实现小白的第一个目标检测任务

适合与我一样的小白上手PaddleX简单预训练模型，可以按照自己的想法完成自己的第一个Paddle项目（或是demo），并且PaddleX的全流程开发，无需担心自己对于PaddlePaddle不熟悉。

# 一、项目背景

在飞桨领航团中学习了一些paddlex项目的流程，尝试写一个目标检测demo，调用预训练模型进行对数据集图片进行训练后，对测试集中图片目标检测。

# 二、数据集简介

PASCAL VOC为图像识别和分类提供了一整套标准化的优秀的数据集，从2005年到2012年每年都会举行一场图像识别challenge。此数据集可以用于图像分类、目标检测、图像分割，非常适合我这样的小白入门尝试，而Paddlex对于VOC十分友好。


In [1]:
# 下载paddlex依赖方便后续使用
!pip install paddlex==2.0rc

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 450kB 12.7MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 15.7MB/s eta 0:00:01
     |████████████████████████████████| 307kB 28.3MB/s eta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=278363 sha256=d41cb18e83c6b79d22148b0e9317c247a6df351fc8395e2737fb26e4b82aac59
  Stored in directory: /home/aistudio/.cache/pip/wheels/fb/44/67/8baa69040569b1edbd7776ec6f82c387663e724908aaa60963
Successfully built pycocotools


In [2]:
#解压数据集
!tar -xf /home/aistudio/data/data37195/VOC2012.tar

In [3]:
#调用paddlex切分数据集----7：2：1的分布训练集、验证集、测试集（在voc文件下）
#paddlex的便利性，直接一步切分数据集
!paddlex --split_dataset --format VOC --dataset_dir VOCdevkit/VOC2012 --val_value 0.2 --test_value 0.1

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:120: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  "Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything."
[08-15 15:24:47 MainThread @logger.py:242] Argv: /opt/conda/envs/python35-paddle120-env/bin/paddlex --split_dataset --format VOC --dataset_dir VOCdevkit/VOC2012 --val_value 0.2 --test_value 0.1
[08-15 15:24:47 MainThread @utils.py:79] WRN paddlepaddle version: 2.1.2. The dynamic graph version of PARL is under development, not fully tested and supported
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default

In [4]:
# 设置参数，对数据中的图片进行处理
# 配置模型使用gpugpu显卡训练（cpu下删除下方代码中的0）
import paddlex as pdx
from paddlex import transforms as T
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0' 
#对读入的数据图片进行处理
train_transforms = T.Compose([
    T.MixupImage(mixup_epoch=250), T.RandomDistort(),
    T.RandomExpand(im_padding_value=[123.675, 116.28, 103.53]), T.RandomCrop(),
    T.RandomHorizontalFlip(), T.BatchRandomResize(
        target_sizes=[320, 352, 384, 416, 448, 480, 512, 544, 576, 608],
        interp='RANDOM'), T.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

eval_transforms = T.Compose([
    T.Resize(
        608, interp='CUBIC'), T.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:120: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  "Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything."


[08-15 15:24:58 MainThread @utils.py:79] WRN paddlepaddle version: 2.1.2. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pyarrow/pandas_compat.py:1027: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  'floating': np.float,
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of

In [5]:
# 读入分配好的数据集，及上方设定的参数
# 按着划分好的list读入
train_dataset = pdx.datasets.VOCDetection(
    data_dir='VOCdevkit/VOC2012',
    file_list='VOCdevkit/VOC2012/train_list.txt',
    label_list='VOCdevkit/VOC2012/labels.txt',
    transforms=train_transforms,
    shuffle=True)

eval_dataset = pdx.datasets.VOCDetection(
    data_dir='VOCdevkit/VOC2012',
    file_list='VOCdevkit/VOC2012/val_list.txt',
    label_list='VOCdevkit/VOC2012/labels.txt',
    transforms=eval_transforms,
    shuffle=False)

2021-08-15 15:25:06 [INFO]	Starting to read file list from dataset...
2021-08-15 15:25:24 [INFO]	11988 samples in file VOCdevkit/VOC2012/train_list.txt
creating index...
index created!
2021-08-15 15:25:24 [INFO]	Starting to read file list from dataset...
2021-08-15 15:25:30 [INFO]	3425 samples in file VOCdevkit/VOC2012/val_list.txt
creating index...
index created!


In [ ]:
#模型参数设定，及使用的预训练模型
#运行该代码进行模型训练
num_classes = len(train_dataset.labels)
model = pdx.models.YOLOv3(num_classes=num_classes, backbone='MobileNetV1')

#模型训练参数设定
model.train(
    num_epochs=200,
    train_dataset=train_dataset,
    train_batch_size=8,
    eval_dataset=eval_dataset,
    learning_rate=0.001 / 8,
    warmup_steps=1000,
    warmup_start_lr=0.0,
    save_interval_epochs=5,
    lr_decay_epochs=[216, 243],
    save_dir='output/yolov3_MobileNetV1')

2021-08-15 15:30:49 [INFO]	[TRAIN] Epoch=1/200, Step=650/1498, loss_xy=6.386331, loss_wh=3.441161, loss_obj=18.503603, loss_cls=11.153825, loss=39.484921, lr=0.000081, time_each_step=0.38s, eta=31:19:27
2021-08-15 15:30:52 [INFO]	[TRAIN] Epoch=1/200, Step=660/1498, loss_xy=6.333994, loss_wh=3.363979, loss_obj=16.449947, loss_cls=11.377101, loss=37.525021, lr=0.000082, time_each_step=0.3s, eta=24:56:13
2021-08-15 15:30:56 [INFO]	[TRAIN] Epoch=1/200, Step=670/1498, loss_xy=7.350976, loss_wh=4.148143, loss_obj=17.111372, loss_cls=10.338604, loss=38.949097, lr=0.000084, time_each_step=0.38s, eta=31:33:35
2021-08-15 15:30:59 [INFO]	[TRAIN] Epoch=1/200, Step=680/1498, loss_xy=9.314291, loss_wh=5.229423, loss_obj=24.398373, loss_cls=16.489712, loss=55.431797, lr=0.000085, time_each_step=0.34s, eta=28:34:47
2021-08-15 15:31:02 [INFO]	[TRAIN] Epoch=1/200, Step=690/1498, loss_xy=6.100159, loss_wh=3.290078, loss_obj=18.768536, loss_cls=9.339243, loss=37.498016, lr=0.000086, time_each_step=0.31s, 

In [ ]:
#模型预测（选取图片查看测试效果）
test_jpg = 'VOCdevkit/VOC2012/JPEGImages/2011_001878.jpg'#可以选取test中的图片测试
model = pdx.load_model('output/yolov3_darknet53/epoch_30')#填写训练后的模型路径，这里是用的我30epoch训练出的模型测试预测

result = model.predict(test_jpg)

# 可视化结果存储在./visualized_test.jpg, 见下图
#pdx.det.visualize(test_jpg, result, threshold=0.3, save_dir='./')----这是PADDLEX文档使用的预测调用但是无法成功运作，我不知道为啥，就换成下面的，
pdx.visualize_det(test_jpg, result, threshold=0.3, save_dir='./')

# 四、效果展示

多训练调参后可以得到更好的效果

![](https://ai-studio-static-online.cdn.bcebos.com/f5a2b419744b4dd5b426ad7da15e5fd43a02c563b07e450b9e941e9fd5462a95)

# 五、总结

自己尝试写的demo，虽然很简单，但是经过paddlex的使用，有了继续探索paddle深度学习框架的兴趣，希望大佬留下宝贵的建议！感谢

# 六、个人简介

吴世君 东北大学秦皇岛分校 测控技术与仪器专业 大二本科生

感兴趣方向：计算机视觉

小小青铜渴望关注

来互关呀~ [me jun](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/791981)
